# 微信好友数据的收集和简单分析

### 1.真的安装了很多包

In [1]:
import os #操作系统模块，文件读写啥的，离不开它
import itchat  #专用的微信个人账号的接口
import numpy as np #numpy，典型数据分析包
import pandas as pd #pandas，典型数据分析包
import matplotlib.pyplot as plt #老朋友，画图用的
import jieba #老朋友
from wordcloud import WordCloud 
from collections import Counter #collections是一个集合模块
import re #正则表达式
import io #读写文件的
#from wordcloud import WordCloud

### 2.以下写了好些个函数，不过不用怕，细细分析下 ，都不难。这是一个用itchat登陆微信的函数

In [2]:
def my_friends(): #登陆并获取好友信息
     #二维码登陆
    itchat.auto_login()
    #获取好友信息 
    friends = itchat.get_friends(update=True) #获取好友信息很简单，是不是
    return friends #返回的变量还是很重要的，因为接下来的函数，就会用到这个变量

### 3.对2中的信息进行抽取与分析，写了个性别统计函数。用来画性别统计图

In [3]:
def my_friends_sex(friends):
   
    #创建一个字典用于存放好友性别信息
    friends_sex = dict()
    #定义好友性别信息字典的key，分别为男性，女性，其他
    male    =  "男性"
    female  =  "女性"
    other   =  "其他"
 
    #遍历列表中每一个好友的信息，     
    for i in friends[1:]: 
        sex = i["Sex"] #找到好友中的sex项
        if sex == 1:
            #字典操作，找到key并为其的值加1
            friends_sex[male] = friends_sex.get(male,0) + 1
        elif sex == 2:
            friends_sex[female] = friends_sex.get(female,0) + 1
        elif sex == 0 :
            friends_sex[other] = friends_sex.get(other,0) + 1
    #打印好友性别信息的字典
    #print (friends_sex)
    #好友总数，从第二个开始是因为第一个好友是自己
    totle = len(friends[1:])
    
    proportion = [float(friends_sex[male])/totle*100,float(friends_sex[female])/totle*100,float(friends_sex[other])/totle*100]
    print (
       u"男性好友：%.2f%% " % (proportion[0])     +'\n' +
       u"女性好友：%.2f%% " % (proportion[1])   +'\n' +
       u"其他：%.2f%% "  % (proportion[2])
       )
    return friends_sex

### 4.对2中的好友信息friends进行抽取，获得好友个性签名。用在词云图制作中

In [4]:
def my_friends_style(friends):
    #创建列表用于存放个性签名
    style = []
    for i in range(len(friends)):
        #每一个好友的信息存放在列表中的字典里，此处获取到
        i = friends[i]
        #得到每个字典的个性签名的key，即Signature
        #strip去除字符串首位的空格，replace去掉英文
        Signature = i['Signature'].strip().replace('span','').replace('class','').replace('emoji','')
        #通过正则表达式将签名中的特殊符号去掉，re.sub则相当于字符串操作中的replace
        rep = re.compile('1f\d+\w*|[<>/=]')
        Signature=rep.sub('',Signature)
        #放入列表
        style.append(Signature)
    #join() 方法用于将序列中的元素以指定的字符连接生成一个新的字符串。
    #此处将所有签名去除特殊符号和英文之后，拼接在一起
    text = ''.join(style)
    #将输出保存到文件，并用结巴来分词
    with io.open('text.txt','a',encoding = 'utf-8') as f:
        wordlist = jieba.cut(text,cut_all=False)
        word_space_split = ' '.join(wordlist)
        f.write(word_space_split)

### 5.写了一个性别统计图函数

In [5]:
def drow_sex(friends_sex):
    #获取饼状图的标签和大小
    labels = []
    sizes = []
    for key in friends_sex:
        labels.append(key)
        sizes.append(friends_sex[key])
    #每块图的颜色，数量不足时会循环使用
    colors = ['red', 'yellow', 'blue']
    #每一块离中心的距离
    explode = (0.1,0,0)
    #autopct='%1.2f%%'百分数保留两位小数点；shadow=True,加阴影使图像更立体
    #startangle起始角度，默认为0°，一般设置为90比较好看
    plt.pie(sizes,explode=explode,labels=labels,colors=colors,autopct='%1.2f%%',shadow=True,startangle=90)
    #设置图像的xy轴一致
    plt.axis('equal')
    #显示颜色和标签对应关系
    plt.legend()
    plt.suptitle(u"微信好友性别统计图")
    #保存到本地，因为show之后会创建空白图层，所以必须在show之前保存
    plt.savefig('好友性别饼状图.png')
    plt.show()

### 6.写了一个词云函数

In [6]:
def wordart():
    big_pic = imread('cat.jpg')
    text = open("text.txt",encoding='utf-8').read()
    wordcloud_cn_x = WordCloud(mask=big_pic,width=800,height=600, background_color= 'white',scale=2.5, font_path="simsun.ttf").generate(text)
    plt.imshow(wordcloud_cn_x)
    plt.axis('off')
    #plt.savefig('词云.png') # 不要用这一句，出来的图不清晰
    wordcloud_cn_x.to_file('词云.jpg') #出图清晰

### 7.主函数。这里仔细看一看，主函数和之前的方法函数，有什么关系。

In [7]:
if __name__ == '__main__':
    friends = my_friends() #首先登陆，并讲返回值存在friends变量里面
    friends_sex = my_friends_sex(friends) #分析统计friends中的性别信息
    drow_sex(friends_sex) ##画性别统计图
    my_friends_style(friends) ##分析friends中的个性签名，不需要返回值，直接被wordart调用
    %pylab inline 
    wordart() 

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.


ExpatError: mismatched tag: line 63, column 4

# 补充：如何将friends的信息存到scv文件中？

这里就要涉及到刚才那么多代码的关键信息了，第2步，获取的friends数据到底是什么格式？里面有哪些字段的信息？运行下面的cell试一试。

In [8]:
friends ##运行之后，你会发现，是一个[]里面套着{}，也就是列表嵌套字典。好吧，有点复杂。但是就是，我们现在需要处理它

[{'UserName': '@87d09dbf33fd9f36a710a865457a8c8f',
  'City': '南京',
  'DisplayName': '',
  'PYQuanPin': 'littlewu',
  'RemarkPYInitial': '',
  'Province': '江苏',
  'KeyWord': 'cnw',
  'RemarkName': '',
  'PYInitial': 'LITTLEW',
  'EncryChatRoomId': '',
  'Alias': '',
  'Signature': '无论做什么，都要从心里做……我所侍奉的乃是主基督。',
  'NickName': 'little 吴',
  'RemarkPYQuanPin': '',
  'HeadImgUrl': '/cgi-bin/mmwebwx-bin/webwxgeticon?seq=627820608&username=@87d09dbf33fd9f36a710a865457a8c8f&skey=@crypt_9e0b370a_cdfbb19a71f46af08d7e5bc302dfd6ff',
  'UniFriend': 0,
  'Sex': 1,
  'AppAccountFlag': 0,
  'VerifyFlag': 0,
  'ChatRoomId': 0,
  'HideInputBarFlag': 0,
  'AttrStatus': 98405,
  'SnsFlag': 273,
  'MemberCount': 0,
  'OwnerUin': 0,
  'ContactFlag': 1,
  'Uin': 1077759741,
  'StarFriend': 0,
  'Statues': 0,
  'MemberList': [],
  'WebWxPluginSwitch': 0,
  'HeadImgFlag': 1,
  'IsOwner': 0},
 {'Uin': 0,
  'UserName': '@95b00f03c99ab1c25ec9094318a6126a523c0737adbaad0d528e86662ecc239c',
  'NickName': '意味',
  'Head

In [11]:
friends[1] #我们看看第一个人的数据结构

{'Uin': 0,
 'UserName': '@95b00f03c99ab1c25ec9094318a6126a523c0737adbaad0d528e86662ecc239c',
 'NickName': '意味',
 'HeadImgUrl': '/cgi-bin/mmwebwx-bin/webwxgeticon?seq=700837042&username=@95b00f03c99ab1c25ec9094318a6126a523c0737adbaad0d528e86662ecc239c&skey=@crypt_9e0b370a_cdfbb19a71f46af08d7e5bc302dfd6ff',
 'ContactFlag': 3,
 'MemberCount': 0,
 'MemberList': [],
 'RemarkName': '晁凡淼-电商17',
 'HideInputBarFlag': 0,
 'Sex': 1,
 'Signature': '街上的红灯绿灯闪烁不停闪烁不停闪烁…',
 'VerifyFlag': 0,
 'OwnerUin': 0,
 'PYInitial': 'YW',
 'PYQuanPin': 'yiwei',
 'RemarkPYInitial': 'CFMDS17',
 'RemarkPYQuanPin': 'chaofanmiaodianshang17',
 'StarFriend': 0,
 'AppAccountFlag': 0,
 'Statues': 0,
 'AttrStatus': 33624189,
 'Province': '江苏',
 'City': '徐州',
 'Alias': '',
 'SnsFlag': 257,
 'UniFriend': 0,
 'DisplayName': '',
 'ChatRoomId': 0,
 'KeyWord': '',
 'EncryChatRoomId': '',
 'IsOwner': 0}

In [12]:
friends[1]['City'] ##我们再看看ta的具体信息，城市。我们还可以看其他数据项。

'徐州'

这样分析之后，心里有数了。我们需要用到第5讲的代码，收集微博数据的时候，用到的csv代码

In [13]:
import csv

In [14]:
path = os.getcwd()+"/wechat.csv"
#path = "/weibo.csv"
# 注意这里的编码是utf-8-sig，如果是utf-8写入的文件是乱码
csvfile = open(path,'a',encoding='utf-8-sig',newline='')
#clearData(csvfile)
#每一次重新运行程序的时候把上一次保存的数据清空 #是重新运行程序#省区手动清空的多余环节
csvfile.seek(0,0)
csvfile.truncate()
writer = csv.writer(csvfile)
#writer.writerow(('username','source','comment'))
writer.writerow(('昵称','备注','性别','省份','城市','签名'))

19

In [15]:
for i in friends[1:]: 
    
    nickname = i['NickName']
    remarkname = i['RemarkName']
    
    #性别汉化一下
    sex = i["Sex"] #找到好友中的sex项
    if sex == 1:
        sex = '爷们'
    elif sex == 2:
        sex = '汉子'
    elif sex == 0 :
        sex = '未知'
                      
    province = i['Province']
    city = i['City']
    
    #签名要数据预处理一下
    signature = i['Signature'].strip().replace('span','').replace('class','').replace('emoji','')
    #通过正则表达式将签名中的特殊符号去掉，re.sub则相当于字符串操作中的replace
    rep = re.compile('1f\d+\w*|[<>/=]')
    signature = rep.sub('',signature)
    
    # 写入csv文件的时候，要按照次序写入，对应中文名称的字段
    writer.writerow((nickname,remarkname,sex,province,city,signature))

In [16]:
csvfile.close()